In [12]:
import pandas as pd
import numpy as np
#from fuzzywuzzy import process
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import cosine_similarity
import joblib

In [13]:
animes = pd.read_csv('data/animes_preprocessed.csv')
ratings = pd.read_csv('data/ratings_preprocessed.csv')

In [14]:
combined = pd.merge(animes, ratings, on='anime_id')
combined

,anime_id,name,genre,type,episodes,rating_x,members,user_id,rating_y
0,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665,17,10
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665,38,10
2,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665,43,4
3,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665,46,9
4,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665,129,9
...,...,...,...,...,...,...,...,...,...
2521252,7808,Zukkoke Knight: Don De La Mancha,"Adventure, Comedy, Historical, Romance",TV,23,6.47,172,1822,6
2521253,7808,Zukkoke Knight: Don De La Mancha,"Adventure, Comedy, Historical, Romance",TV,23,6.47,172,48766,0
2521254,7808,Zukkoke Knight: Don De La Mancha,"Adventure, Comedy, Historical, Romance",TV,23,6.47,172,49127,6
2521255,7808,Zukkoke Knight: Don De La Mancha,"Adventure, Comedy, Historical, Romance",TV,23,6.47,172,51693,7


In [15]:
anime_matrix = combined.pivot_table(index='name',columns='user_id',values='rating_y').fillna(0)
anime_matrix

user_id,5,7,17,38,43,46,54,123,129,139,...,73417,73422,73457,73460,73476,73491,73499,73502,73503,73507
name,,,,,,,,,,,,,,,,,,,,,
.hack//Roots,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
.hack//Sign,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,...,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
.hack//Tasogare no Udewa Densetsu,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0-sen Hayato,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
009-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
gdgd Fairies 2,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
iDOLM@STER Xenoglossia,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
s.CRY.ed,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
anime_titles = anime_matrix.reset_index()
anime_titles = anime_titles['name'].to_frame()
anime_titles

,name
0,.hack//Roots
1,.hack//Sign
2,.hack//Tasogare no Udewa Densetsu
3,0-sen Hayato
4,009-1
...,...
3394,gdgd Fairies 2
3395,iDOLM@STER Xenoglossia
3396,s.CRY.ed
3397,xxxHOLiC


In [17]:
def get_index(anime_name):
    record = anime_titles[anime_titles['name'] == anime_name]
    anime_titles = list(record.index) 
    return anime_titles[0]

In [18]:
anime_matrix_sparse = csr_matrix(anime_matrix.values)

In [19]:
cosine_sim = cosine_similarity(anime_matrix_sparse)
cosine_sim

array([[1.        , 0.64155625, 0.63758223, ..., 0.23736487, 0.26384838,
        0.21641249],
       [0.64155625, 1.        , 0.64244157, ..., 0.33054178, 0.31590918,
        0.25387812],
       [0.63758223, 0.64244157, 1.        , ..., 0.2600452 , 0.25198586,
        0.20350851],
       ...,
       [0.23736487, 0.33054178, 0.2600452 , ..., 1.        , 0.231488  ,
        0.18768972],
       [0.26384838, 0.31590918, 0.25198586, ..., 0.231488  , 1.        ,
        0.84964157],
       [0.21641249, 0.25387812, 0.20350851, ..., 0.18768972, 0.84964157,
        1.        ]])

In [20]:
filename = 'cosine_sim.sav'
joblib.dump(cosine_sim, filename)

['cosine_sim.sav']

In [21]:
def item_based_recomm(anime_name):    
    recommender_df = pd.DataFrame(cosine_sim, 
                                  columns=anime_matrix.index,
                                  index=anime_matrix.index)

    ## Item Rating Based Cosine Similarity
    cosine_df = pd.DataFrame(recommender_df[anime_name])
    cosine_df.reset_index(level=0, inplace=True)
    cosine_df.columns = ['name','cosine_sim']
    
    cosine_df = pd.merge(animes, cosine_df, on='name')
    cosine_df.sort_values(by='cosine_sim',ascending=False,inplace=True)
    cosine_df = cosine_df.iloc[1:,]
    
    cosine_df.reset_index(inplace=True,drop=True)

    return cosine_df.head(20)

In [22]:
yo = item_based_recomm('Berserk')
yo

,anime_id,name,genre,type,episodes,rating,members,cosine_sim
0,1,Cowboy Bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",TV,26,8.82,486824,0.532546
1,889,Black Lagoon,"Action, Seinen",TV,12,8.17,332562,0.521915
2,1818,Claymore,"Action, Adventure, Demons, Fantasy, Shounen, S...",TV,26,7.92,316853,0.520915
3,270,Hellsing,"Action, Horror, Seinen, Supernatural, Vampire",TV,13,7.64,308995,0.503806
4,6,Trigun,"Action, Comedy, Sci-Fi",TV,26,8.32,283069,0.503411
5,205,Samurai Champloo,"Action, Adventure, Comedy, Historical, Samurai...",TV,26,8.50,390076,0.499530
6,1519,Black Lagoon: The Second Barrage,"Action, Seinen",TV,12,8.30,203233,0.497601
7,30,Neon Genesis Evangelion,"Action, Dementia, Drama, Mecha, Psychological,...",TV,26,8.32,461946,0.489372
8,245,Great Teacher Onizuka,"Comedy, Drama, School, Shounen, Slice of Life",TV,43,8.77,268487,0.487013
9,1535,Death Note,"Mystery, Police, Psychological, Supernatural, ...",TV,37,8.71,1013917,0.482112
